In [1]:
from __future__ import print_function
from pathlib import Path
from random import shuffle
import random

import argparse
import copy
import numpy as np
import sys
import tensorflow as tf
import scipy.sparse as sp

from deeplp.models.data_prep import create_weighted_graph, load_data
from deeplp.models.data_prep import prepare_data, random_unlabel, calc_masks
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.models.utils import accuracy, indices_to_vec
from deeplp.models.data_prep import select_features


/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Spectral Clustering

In [ ]:
from sklearn.cluster import SpectralClustering as SC

In [ ]:
ress = []
for data in ['cora','dblp','flickr','imdb','industry']:
    res = []
    for seed in range(10):
        true_labels, features, edge_features, node_features, graph \
        = load_data(data,'flip',directed=1,confidence=0)


        labeled_indices, unlabeled_indices = \
            random_unlabel(true_labels,0.99,features,
                           seed=seed,confidence=0)
        num_nodes, num_classes = true_labels.shape

        labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)

        sc = SC(n_clusters=labels.shape[1],affinity='precomputed')
        pred = sc.fit_predict(graph.toarray())
        acc = np.mean(pred[unlabeled_indices] == np.argmax(true_labels,axis=1)[unlabeled_indices])
        res.append(acc)
        print(acc)
    mean_acc = np.mean(np.array(res))
    print(data,mean_acc)
    ress.append(mean_acc)

-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!


/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/scipy/sparse/csgraph/_laplacian.py:72: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if normed and (np.issubdtype(csgraph.dtype, int)


0.11666666666666667
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.11666666666666667
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.11463414634146342
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.12032520325203253
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.11504065040650406
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.11666666666666667
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.1516260162601626
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.275609756097561
-----------cora-----------
Loading labels...
Loading

In [ ]:
from sklearn.manifold import spectral_embedding as SE

In [43]:
se = SE(graph.toarray())

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/scipy/sparse/csgraph/_laplacian.py:72: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if normed and (np.issubdtype(csgraph.dtype, int)


In [44]:
se.shape

(2485, 8)

## Laplacian Eigenmap

In [45]:
from sklearn.manifold import spectral_embedding as SE

In [47]:
ress = []
for data in ['cora','dblp','flickr','imdb','industry']:
    true_labels, features, edge_features, node_features, graph \
    = load_data(data,'flip',directed=1,confidence=0)

    se = SE(graph.toarray())
    np.savetxt(f'datasets/flip/{data}/embed_laplacian_eigen.csv',se,delimiter=' ')
    

-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!


/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/scipy/sparse/csgraph/_laplacian.py:72: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if normed and (np.issubdtype(csgraph.dtype, int)


-----------dblp-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
-----------imdb-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
-----------industry-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!


## LINE

In [1]:
for data in ['cora','dblp','flickr','imdb','industry']:
    print(data)
    !./LINE/linux/a.out -train datasets/flip/{data}/Wdir_line.txt -output datasets/flip/{data}/embed_line.txt -binary 0 -size 200 -order 2 -negative 5 -samples 100 -rho 0.025 -threads 20

cora
--------------------------------
Order: 2
Samples: 100M
Negative: 5
Dimension: 200
Initial rho: 0.025000
--------------------------------
Number of edges: 10138          
Number of vertices: 2485          
--------------------------------
Rho: 0.000050  Progress: 99.810%
Total time: 1602.395730
dblp
--------------------------------
Order: 2
Samples: 100M
Negative: 5
Dimension: 200
Initial rho: 0.025000
--------------------------------
Number of edges: 38376          
Number of vertices: 5314          
--------------------------------
Rho: 0.000050  Progress: 99.810%
Total time: 1643.093295
flickr
--------------------------------
Order: 2
Samples: 100M
Negative: 5
Dimension: 200
Initial rho: 0.025000
--------------------------------
Number of edges: 957960          
Number of vertices: 7971          
--------------------------------
Rho: 0.000050  Progress: 99.810%
Total time: 1647.152847
imdb
--------------------------------
Order: 2
Samples: 100M
Negative: 5
Dimension: 200
Initia

## node2vec

In [2]:
for data in ['flickr',]:
    print(data)
    !python node2vec/src/main.py --input datasets/flip/{data}/Wdir_node2vec.txt --output datasets/flip/{data}/embed_node2vec.txt

flickr
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


## Log Reg from Embeddings

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import LogisticRegressionCV as LRCV

In [12]:
accuracies = np.zeros((6,5,10))
#'cora','dblp','flickr','imdb',
for i,data in enumerate(['cora','dblp','flickr','imdb','industry']):
    laplacian_eigen = np.loadtxt(f'datasets/flip/{data}/embed_laplacian_eigen.csv',delimiter=' ')
    deepwalk = pd.read_csv(f'datasets/flip/{data}/embed_deepwalk_{data}.txt',skiprows=[0],delimiter=' ',index_col=0,header=None)
    deepwalk = deepwalk.sort_index().as_matrix()
    line = pd.read_csv(f'datasets/flip/{data}/embed_line.txt',skiprows=[0],delimiter=' ',index_col=0,header=None)
    line = line.sort_index().iloc[:,:-1].as_matrix()    
    node2vec = pd.read_csv(f'datasets/flip/{data}/embed_node2vec.txt',skiprows=[0],delimiter=' ',index_col=0,header=None)
    node2vec = node2vec.sort_index().iloc[:,:-1].as_matrix()
    

    for seed in range(10):
        true_labels, features, edge_features, node_features, graph \
        = load_data(data,'flip',directed=1,confidence=0)

        labeled_indices, unlabeled_indices = \
            random_unlabel(true_labels,0.99,features,
                           seed=seed,confidence=0)
        num_nodes, num_classes = true_labels.shape

        labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)
        if data == 'industry':
            lr=LR()
        else:
            lr=LRCV(Cs=5)
    
        # 0: LabelProp Iter
        lp = LP()
        unlabeled_pred = lp.iter_sp(labels,
                             graph,
                             is_labeled,
                             10,
                             unlabeled_indices)
        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        accuracies[0,i,seed] = acc
        print("baseline",acc)
        
        
        # 1: LabelProp Closed
        unlabeled_pred = lp.closed_sp(labels,
                                   graph,
                                   labeled_indices,
                                   unlabeled_indices)
        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        accuracies[1,i,seed] = acc
        print("baseline closed",acc)
        
        # 2: Laplacian Eigen
        lr.fit(laplacian_eigen[labeled_indices],np.argmax(true_labels,axis=1)[labeled_indices])
        pred = lr.predict(laplacian_eigen[unlabeled_indices])
        acc = np.mean(pred == np.argmax(true_labels,axis=1)[unlabeled_indices])
        accuracies[2,i,seed] = acc
        print('laplacian_eigen',acc)
        
        # 3: Deepwalk
        lr.fit(deepwalk[labeled_indices],np.argmax(true_labels,axis=1)[labeled_indices])
        pred = lr.predict(deepwalk[unlabeled_indices])
        acc = np.mean(pred == np.argmax(true_labels,axis=1)[unlabeled_indices])
        accuracies[3,i,seed] = acc
        print('deepwalk',acc)
        
        # 4: LINE
        lr.fit(line[labeled_indices],np.argmax(true_labels,axis=1)[labeled_indices])
        pred = lr.predict(line[unlabeled_indices])
        acc = np.mean(pred == np.argmax(true_labels,axis=1)[unlabeled_indices])
        accuracies[4,i,seed] = acc
        print('line',acc)
        
        # 5: Node2Vec
        lr.fit(node2vec[labeled_indices],np.argmax(true_labels,axis=1)[labeled_indices])
        pred = lr.predict(node2vec[unlabeled_indices])
        acc = np.mean(pred == np.argmax(true_labels,axis=1)[unlabeled_indices])
        accuracies[5,i,seed] = acc
        print('node2vec',acc)


-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.4479674796747967
baseline closed 0.16504065040650406
laplacian_eigen 0.23536585365853657
deepwalk 0.540650406504065
line 0.3178861788617886
node2vec 0.4573170731707317
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.676829268292683
baseline closed 0.4658536585365854


/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


laplacian_eigen 0.4410569105691057
deepwalk 0.4658536585365854
line 0.3573170731707317
node2vec 0.4902439024390244
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.6760162601626016
baseline closed 0.6321138211382114
laplacian_eigen 0.46422764227642277
deepwalk 0.4508130081300813
line 0.29227642276422766
node2vec 0.5138211382113821
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.5983739837398374
baseline closed 0.6089430894308943
laplacian_eigen 0.45203252032520325
deepwalk 0.4410569105691057
line 0.2524390243902439
node2vec 0.5650406504065041
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.6707317073170732
baseline closed 0.46178861788617886
laplacian_eigen 0.5711382113821138
deepwalk 0.4813008130081301
line 0.3308943089430894
node2vec 0.6272357723577235
--------

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


laplacian_eigen 0.5146341463414634
deepwalk 0.6508130081300812
line 0.3361788617886179
node2vec 0.651219512195122
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.6020325203252033
baseline closed 0.3878048780487805
laplacian_eigen 0.4947154471544715
deepwalk 0.4695121951219512
line 0.2491869918699187
node2vec 0.4630081300813008
-----------dblp-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.6684410646387833
baseline closed 0.4950570342205323
laplacian_eigen 0.564638783269962
deepwalk 0.59106463878327
line 0.3967680608365019
node2vec 0.5986692015209125
-----------dblp-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.7326996197718632
baseline closed 0.5444866920152092
laplacian_eigen 0.5665399239543726
deepwalk 0.5796577946768061
line 0.40247148288973383
node2vec 0.6220532319391635
-----------dblp

Done!
baseline 0.5945945945945946
baseline closed 0.4927927927927928
laplacian_eigen 0.5072072072072072
deepwalk 0.5072072072072072
line 0.5072072072072072
node2vec 0.5072072072072072
-----------imdb-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.5099099099099099
baseline closed 0.4918918918918919
laplacian_eigen 0.6792792792792792
deepwalk 0.49099099099099097
line 0.49099099099099097
node2vec 0.49099099099099097
-----------imdb-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.5207207207207207
baseline closed 0.4918918918918919
laplacian_eigen 0.5171171171171172
deepwalk 0.5513513513513514
line 0.572972972972973
node2vec 0.5306306306306307
-----------industry-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
baseline 0.3185462319615179
baseline closed 0.30732228754676644
laplacian_eigen 0.3003741314804917
deepwalk 0.20256

In [13]:
np.mean(accuracies,2)

array([[0.61418699, 0.70087452, 0.66822963, 0.55072072, 0.2130946 ],
       [0.44012195, 0.57547529, 0.45222405, 0.51306306, 0.20486371],
       [0.4604878 , 0.56904943, 0.63752376, 0.59018018, 0.21635489],
       [0.48264228, 0.65572243, 0.79993664, 0.52018018, 0.2171566 ],
       [0.30934959, 0.44648289, 0.79908757, 0.51432432, 0.19786211],
       [0.53345528, 0.67038023, 0.81350906, 0.55477477, 0.21747728]])

In [15]:
accs10=[]
accs50=[]
accs100=[]
accsclosed=[]
for i,data in enumerate(['citeseer']):
#     laplacian_eigen = np.loadtxt(f'datasets/flip/{data}/embed_laplacian_eigen.csv',delimiter=' ')
#     deepwalk = pd.read_csv(f'datasets/flip/{data}/embed_deepwalk_{data}.txt',skiprows=[0],delimiter=' ',index_col=0,header=None)
#     deepwalk = deepwalk.sort_index().as_matrix()
#     line = pd.read_csv(f'datasets/flip/{data}/embed_line.txt',skiprows=[0],delimiter=' ',index_col=0,header=None)
#     line = line.sort_index().iloc[:,:-1].as_matrix()    
#     node2vec = pd.read_csv(f'datasets/flip/{data}/embed_node2vec.txt',skiprows=[0],delimiter=' ',index_col=0,header=None)
#     node2vec = node2vec.sort_index().iloc[:,:-1].as_matrix()

    for seed in range(10):
        true_labels, features, edge_features, node_features, graph \
        = load_data(data,'linqs',directed=1,confidence=0)

        labeled_indices, unlabeled_indices = \
            random_unlabel(true_labels,0.99,features,
                           seed=seed,confidence=0)
        num_nodes, num_classes = true_labels.shape

        labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)

    
#         # 0: LabelProp Iter
        lp = LP()
        unlabeled_pred = lp.iter_sp(labels,
                             graph,
                             is_labeled,
                             10,
                             unlabeled_indices)
        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        print("baseline",acc)
        accs10.append(acc)
        unlabeled_pred = lp.iter_sp(labels,
                             graph,
                             is_labeled,
                             50,
                             unlabeled_indices)
        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        print("baseline",acc)
        accs50.append(acc)
        unlabeled_pred = lp.iter_sp(labels,
                             graph,
                             is_labeled,
                             100,
                             unlabeled_indices)
        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        print("baseline",acc)
        accs100.append(acc)
        
        unlabeled_pred = lp.closed_sp(labels,
                                   graph,
                                   labeled_indices,
                                   unlabeled_indices)
        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        print("baseline",acc)
        accsclosed.append(acc)

        

-----------citeseer-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
baseline 0.5196360153256705
baseline 0.5617816091954023
baseline 0.5632183908045977
baseline 0.5076628352490421
-----------citeseer-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
baseline 0.5847701149425287
baseline 0.6068007662835249
baseline 0.6115900383141762
baseline 0.5646551724137931
-----------citeseer-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
baseline 0.42432950191570884
baseline 0.46695402298850575
baseline 0.4774904214559387
baseline 0.45737547892720304
-----------citeseer-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
baseline 0.5421455938697318
baseline 0.5675287356321839
baseline 0.553639846743295
baseline 0.49090038314176243
-

In [16]:
np.mean(accs10),np.mean(accs50),np.mean(accs100),np.mean(accsclosed)

(0.5272509578544061, 0.543103448275862, 0.5414750957854405, 0.5007662835249043)

In [1]:
accuracies

NameError: name 'accuracies' is not defined